<a href="https://colab.research.google.com/github/bagudzz/uas_CNN_Alexnet_C/blob/main/uas-DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UAS Deepl Learning, CNN using Alexnet
Bagus kustiono.O / 202211420017

In [1]:
!pip -q install gradio
!pip -q install torch torchvision

Load Model & Preprocesing

In [2]:
import torch
import torch.nn as nn
from torchvision.models import alexnet, AlexNet_Weights
from torchvision import transforms
from PIL import Image
import gradio as gr

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

# build model
model = alexnet(weights=AlexNet_Weights.DEFAULT)
in_f = model.classifier[6].in_features
model.classifier[6] = nn.Linear(in_f, 2)

# load your weights
ckpt = torch.load("weights/alexnet_catsdogs.pth", map_location=device)
model.load_state_dict(ckpt["model_state"])
model.to(device).eval()

class_to_idx = ckpt.get("class_to_idx", {"cat": 0, "dog": 1})
idx_to_class = {v: k for k, v in class_to_idx.items()}

@torch.no_grad()
def predict(pil_img: Image.Image):
    x = preprocess(pil_img.convert("RGB")).unsqueeze(0).to(device)
    probs = torch.softmax(model(x), dim=1)[0].detach().cpu().tolist()
    return {idx_to_class[i]: float(probs[i]) for i in range(len(probs))}


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 143MB/s]


UI Gradio & Launch

In [3]:
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload gambar"),
    outputs=gr.Label(num_top_classes=2, label="Prediksi"),
    title="Cats vs Dogs - AlexNet (Colab + Gradio)",
    description="Demo klasifikasi kucing vs anjing pakai AlexNet pretrained."
)

demo.launch(share=True, debug=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://65499e68c5fc34f3a8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
